## Linear Regression (continued)

Last time, we saw that linear regression could be written as
$$ y = \beta_0 + \beta_1x_1 + \beta_2x_2 ... + \epsilon_i,  \epsilon_i \sim N(0, \sigma^2) $$

In addition, we saw different ways that you could use to estimate the coefficients, $\beta$. However, we did not really touch upon the predictors (otherwise known as covariates, independent variables, features, etc). In practice, as long as the x values are numeric, they can be included in the model. 

#### Example:

For example, if we have some variable $y$, and we want to use $age = x_1$ and $gender = x_2$ where gender is equal to 1 for male and 0 for female:

$$ y = \beta_0 + \beta_1x_1^2 + \beta_2x_2 $$

$$y = \beta_0 + \beta_1 \log(x_1) + \beta x_2$$

$$ y = \beta_0 + \beta_1x_1*x_2 + \beta_2x_1 + \beta_3x_2 $$

#### this is no longer linear:

$$ y = \beta_0 + \beta_1x_1 + \beta_2^2x_2 $$

### Categorical Variables:

Many times, you will have what are known as *categorical* variables, meaning that they take several discrete values. For example, gender and race are common examples of *categorical* variables. How do we use these in linear regression (and other models?). We can create dummy variables/indicator variables! 

If the categorical variable is binary, we can simply create a variable called gender, where x=1 is equal to male or female, and x=0 is the opposite. However, if we have a categorical variable with $k$ possible conditions, we can make $k-1$ columns to represent that. 

#### Why $k-1$?

last time, we fit a model that was:

`num_lab_procedures = intercept + num_medications`

Let's add race!

In [85]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
diabetes_df = pd.read_csv("../Assignments/hw2/data/diabetic_data.csv")

In [87]:
y = diabetes_df['num_lab_procedures'].values
x = np.column_stack((np.ones((diabetes_df.shape[0])), diabetes_df['num_medications'].values))

In [88]:
unique_race = diabetes_df.groupby(['encounter_id'])['race'].first()

In [90]:
dummies = pd.get_dummies(unique_race)
dummies.head()

,?,AfricanAmerican,Asian,Caucasian,Hispanic,Other
encounter_id,,,,,,
12522,0,0,0,1,0,0
15738,0,0,0,1,0,0
16680,0,0,0,1,0,0
28236,0,1,0,0,0,0
35754,0,0,0,1,0,0


In [67]:
dummies.apply(sum, axis = 1).value_counts()

1    101766
dtype: int64

In [92]:
x = np.column_stack((x, dummies.values))
x

array([[ 1.,  1.,  0., ...,  1.,  0.,  0.],
       [ 1., 18.,  0., ...,  1.,  0.,  0.],
       [ 1., 13.,  0., ...,  1.,  0.,  0.],
       ...,
       [ 1.,  9.,  0., ...,  1.,  0.,  0.],
       [ 1., 21.,  0., ...,  1.,  0.,  0.],
       [ 1.,  3.,  0., ...,  1.,  0.,  0.]])

In [84]:
x # Design matrix 

array([[ 1.,  1.,  0.,  0.,  1.,  0.],
       [ 1., 18.,  0.,  0.,  1.,  0.],
       [ 1., 13.,  0.,  0.,  1.,  0.],
       ...,
       [ 1.,  9.,  0.,  0.,  1.,  0.],
       [ 1., 21.,  0.,  0.,  1.,  0.],
       [ 1.,  3.,  0.,  0.,  1.,  0.]])

##### Quick Note on Design Matrices (model matrix)

A *design matrix*, or model matrix, must have 1 row per observation, where each column is a variable that characterizes the observation. This is usually known as having *tidy* data. Almost all machine learning models assume this format.

In [74]:
np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(x), x)), np.transpose(x)), y)

array([ 2.05673779e+03,  5.77584673e-01, -2.08963037e+03, -2.13861914e+03,
       -2.16849365e+03, -2.14056641e+03, -2.06651150e+03, -2.06993795e+03])

In [75]:
model = sm.OLS(y, x).fit()

In [76]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     1343.
Date:                Tue, 12 Feb 2019   Prob (F-statistic):               0.00
Time:                        11:20:13   Log-Likelihood:            -4.4371e+05
No. Observations:              101766   AIC:                         8.874e+05
Df Residuals:                  101759   BIC:                         8.875e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         28.5211      0.180    158.387      0.0

#### Multicollinearity:

When one or more of the predictors in a linear model can be written as a linear combination of other predictors, this is known as *perfect multicollinearity*. In this case, it is not possible to obtain the correct estimates for the model coefficients in linear regression. If some of the predictors are highly correlated with themselves, this also results in multicollinearity.

Oftentimes, when you have multicollinearity, you will get extremely large standard errors in your estimates, and many of the hypothesis tests will not reject, even when the model as a whole is significant.

#### Solution: set 1 base case

In [93]:
x = np.column_stack((np.ones((diabetes_df.shape[0])), diabetes_df['num_medications'].values))

In [94]:
unique_race = diabetes_df.groupby(['encounter_id'])['race'].first()
dummies = pd.get_dummies(unique_race)
dummies.head()

,?,AfricanAmerican,Asian,Caucasian,Hispanic,Other
encounter_id,,,,,,
12522,0,0,0,1,0,0
15738,0,0,0,1,0,0
16680,0,0,0,1,0,0
28236,0,1,0,0,0,0
35754,0,0,0,1,0,0


In [80]:
x = np.column_stack((x, dummies.iloc[:, 1:5].values))

In [81]:
np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(x), x)), np.transpose(x)), y)

array([33.67666855,  0.6538995 ,  0.35181106, -1.15595682, -1.49197443,
       -0.05784078])

In [82]:
model = sm.OLS(y, x).fit()

In [83]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     1612.
Date:                Tue, 12 Feb 2019   Prob (F-statistic):               0.00
Time:                        11:23:50   Log-Likelihood:            -4.4371e+05
No. Observations:              101766   AIC:                         8.874e+05
Df Residuals:                  101760   BIC:                         8.875e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         33.6767      0.328    102.522      0.0

#### Covariates as controls

One interpretation of covariates in the linear regression model is that they are controls. If they are included in the model, it means that they are controlled for. It is possible to say, holding values of all of the other predictors constant, the an increase in the response variable $y$ is related to a $\beta$ amount of increase in the variable, or vice versa.

$$ y = \beta_0 + \beta_1x_1^2 + \beta_2x_2 $$

### Interaction variables

Sometimes, it may be useful to use interaction variables. In regular regression, where no interactions are specified, the coefficients $\beta$ represent the unique impact that each covariate $x$ has on the response $y$. However, it is often the case that we believe that the effect on $y$ for different values of $x$ also depend on interactions between the $x$ variables.


#### Example:

In the above regression, we fit `num_lab_procedures = intercept + num_medications + race1 + race2...`

However, what if we believe that, the assocation between the number of lab procudures and the number of medications is actually different depending on your race? We could capture that in the following way:

`num_lab_procedures = intercept + num_medications + race1 + num_medications * race1 + race2 + num_medications * race2...`

Now, the effect of the number of medications on the number of lab procedures is decomposed into the part that is not specific to race, and those that are. This is a fairly common procedure in statistical analysis when there is some suspected dependence between the predictors.

## Why not include everything?

Why, if I had a ton of data, wouldn't I just use every variable?

If I had 20 variables, and then did pairwise interactions between them, I would have 20 + (20 * 19 /2) + 1(intercept) terms in my model. That's 211 variables. Why not?

Last time, we discussed the the $R^2$ was used in model-checking in a traditional regression analysis approach. 

$$var(y) = var(\boldsymbol{\beta}^T\textbf{x}) + var(\epsilon)$$

$$R^2 = \frac{var(\boldsymbol{\beta}^T\textbf{x})}{var(y)} = \frac{\Sigma_{i=1}^n(\hat{y}_i - \bar{y})^2}{\Sigma_{i=1}^n(y_i - \bar{y})} $$

However, we can always make $R^2$ larger if we just keep adding terms! This is why $R^2$ is not the best choice by itself for model validation.

### Curse of Dimensionality

![](./assets/cod.png)

For a uniformly random point in the box in d dimensions with length 2 in each dimension, what is the probability that the random vector is in the unit ball in d dimensions?

### Exercise: 

Plot the ratio of the volume of an $n$-ball (a ball in n-dimensions) with radius 1 versus the volume of a hypercube with a side length of 2 for dimensions 1 through 100

### Volume of an *n*-ball

$$V_n(R) = \frac{\pi^{n/2}}{\Gamma(\frac{n}{2} + 1)}R^n$$

where $\Gamma$ is the generalized gamma function 

In [12]:
import math
from scipy.special import gamma
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# math.pi
# the gamma function has been imported as gamma
def get_hypersphere_volume(dimension, radius):
    pass

In [6]:
def get_hypercube_volume(dimension, length):
    pass 

![](./assets/xkcd.png)